Project 2 - Group 2

In [1]:
#Dependencies
import pandas as pd
import numpy as np
import json
import requests
import time
from pandas import json_normalize
import os
import csv

#Import API Key
# from config import tm_api_key

TicketMaster Section

In [2]:
# #Making call to TicketMaster API

# tm_url = "https://app.ticketmaster.com/discovery/v2/events.json?classificationName=music&countryCode=US"

# tm_query_url = f"{tm_url}&apikey={tm_api_key}"

# tm_response = requests.get(tm_query_url)
# tm_json = tm_response.json()

In [3]:
# print(tm_json)


In [4]:
#Converting Ticketmaster json into 
# tm_events = tm_json['_embedded']['events']
# tm_df = json_normalize(tm_events)
# tm_df.head()

MTV Section

In [5]:
#Reading in mtv genre information and converting it into dataframes
mtv1_path = "data/mtv1.csv"
mtv2_path = "data/mtv2.csv"
mtv3_path = "data/mtv3.csv"
mtv4_path = "data/mtv4.csv"

mtv1_data = pd.read_csv(mtv1_path, skipinitialspace = True)
mtv2_data = pd.read_csv(mtv2_path, skipinitialspace = True)
mtv3_data = pd.read_csv(mtv3_path, skipinitialspace = True)
mtv4_data = pd.read_csv(mtv4_path, skipinitialspace = True)

#Combining all of the mtv df into one
mtv_total_data = pd.concat([mtv1_data, mtv2_data, mtv3_data, mtv4_data])
mtv_total_data.head()

,name,facebook,twitter,website,genre,mtv
0,Adele,http://www.facebook.com/9770929278,http://www.twitter.com/officialadele,NaN,Pop,http://www.mtv.com/artists/adele/biography
1,Joey + Rory,http://www.facebook.com/15044507815,http://www.twitter.com/joeyandrory,NaN,Country,http://www.cmt.com/artists/joey-rory/biography
2,Draaco Aventura,http://www.facebook.com/856796091053581,http://www.twitter.com/DraacoAventura,http://www.bandpage.com/draacoaventura,Pop Latino,http://www.mtv.com/artists/draaco-aventura/bio...
3,Justin Bieber,http://www.facebook.com/309570926875,http://www.twitter.com/justinbieber,http://www.justinbiebermusic.com,Pop,http://www.mtv.com/artists/justin-bieber/biogr...
4,Peer van Mladen,http://www.facebook.com/264487966,http://www.twitter.com/Predrag_Jugovic,http://pejaintergroup.eu/Peer_van_Mladen.html,House,http://www.mtv.com/artists/peer-van-mladen/bio...


In [6]:
mtv_total_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8344 entries, 0 to 1016
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      8336 non-null   object
 1   facebook  3222 non-null   object
 2   twitter   3030 non-null   object
 3   website   2477 non-null   object
 4   genre     5118 non-null   object
 5   mtv       8344 non-null   object
dtypes: object(6)
memory usage: 456.3+ KB


In [7]:
#Dropping the unused columns
mtv_genres = mtv_total_data.drop(columns=["facebook", "twitter","website", "mtv"])

In [8]:
mtv_genres.info()
#Stripping the blank space
mtv_genres["clean"] = mtv_genres["name"].str.strip()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8344 entries, 0 to 1016
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    8336 non-null   object
 1   genre   5118 non-null   object
dtypes: object(2)
memory usage: 195.6+ KB


In [9]:
mtv_genres.head(10)

,name,genre,clean
0,Adele,Pop,Adele
1,Joey + Rory,Country,Joey + Rory
2,Draaco Aventura,Pop Latino,Draaco Aventura
3,Justin Bieber,Pop,Justin Bieber
4,Peer van Mladen,House,Peer van Mladen
5,Chris Janson,Country,Chris Janson
6,One Direction,Pop,One Direction
7,Drake,Hip-Hop/Rap,Drake
8,Carrie Underwood,Country,Carrie Underwood
9,Ed Sheeran,Singer/Songwriter,Ed Sheeran


In [10]:
#Using the cleaned name and deleting the one with extraneous spaces
mtv_genres_rename = mtv_genres.rename(columns = {"name" : "name_spaces", "clean": "name"})
mtv_genres_final = mtv_genres_rename.drop(columns = ["name_spaces"])


mtv_genres_final.to_csv("data/mtv_genres_final.csv")

In [11]:
#Grouping by genre
genre_group = mtv_genres_final.groupby(["genre"]).count()
genre_group

#Genre Totals exported to csv
genre_group.to_csv("data/genre_totals.csv")

Hot100/Spotify

In [12]:
hot100_path = "data/hot100.csv"
hotstuff_path = "data/hotstuff.csv"

hot100_data = pd.read_csv(hot100_path)
hotstuff_data = pd.read_csv(hotstuff_path)

In [13]:
hot100_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29503 entries, 0 to 29502
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   SongID                     29503 non-null  object 
 1   Performer                  29503 non-null  object 
 2   Song                       29503 non-null  object 
 3   spotify_genre              27903 non-null  object 
 4   spotify_track_id           24397 non-null  object 
 5   spotify_track_preview_url  14491 non-null  object 
 6   spotify_track_duration_ms  24397 non-null  float64
 7   spotify_track_explicit     24397 non-null  object 
 8   spotify_track_album        24391 non-null  object 
 9   danceability               24334 non-null  float64
 10  energy                     24334 non-null  float64
 11  key                        24334 non-null  float64
 12  loudness                   24334 non-null  float64
 13  mode                       24334 non-null  flo

In [14]:
hotstuff_data.head()


,url,WeekID,Week Position,Song,Performer,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart
0,http://www.billboard.com/charts/hot-100/1965-0...,7/17/1965,34,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,45.0,34,4
1,http://www.billboard.com/charts/hot-100/1965-0...,7/24/1965,22,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,34.0,22,5
2,http://www.billboard.com/charts/hot-100/1965-0...,7/31/1965,14,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,22.0,14,6
3,http://www.billboard.com/charts/hot-100/1965-0...,8/7/1965,10,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,14.0,10,7
4,http://www.billboard.com/charts/hot-100/1965-0...,8/14/1965,8,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,10.0,8,8


In [15]:
hotstuff_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327895 entries, 0 to 327894
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   url                     327895 non-null  object 
 1   WeekID                  327895 non-null  object 
 2   Week Position           327895 non-null  int64  
 3   Song                    327895 non-null  object 
 4   Performer               327895 non-null  object 
 5   SongID                  327895 non-null  object 
 6   Instance                327895 non-null  int64  
 7   Previous Week Position  295941 non-null  float64
 8   Peak Position           327895 non-null  int64  
 9   Weeks on Chart          327895 non-null  int64  
dtypes: float64(1), int64(4), object(5)
memory usage: 25.0+ MB


In [16]:
hot100_data.head()

,SongID,Performer,Song,spotify_genre,spotify_track_id,spotify_track_preview_url,spotify_track_duration_ms,spotify_track_explicit,spotify_track_album,danceability,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,spotify_track_popularity
0,-twistin'-White Silver SandsBill Black's Combo,Bill Black's Combo,-twistin'-White Silver Sands,[],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,¿Dònde Està Santa Claus? (Where Is Santa Claus...,Augie Rios,¿Dònde Està Santa Claus? (Where Is Santa Claus?),['novelty'],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,......And Roses And RosesAndy Williams,Andy Williams,......And Roses And Roses,"['adult standards', 'brill building pop', 'eas...",3tvqPPpXyIgKrm4PR9HCf0,https://p.scdn.co/mp3-preview/cef4883cfd1e0e53...,166106.0,False,The Essential Andy Williams,0.154,...,-14.063,1.0,0.0315,0.91100,0.000267,0.112,0.150,83.969,4.0,38.0
3,...And Then There Were DrumsSandy Nelson,Sandy Nelson,...And Then There Were Drums,"['rock-and-roll', 'space age pop', 'surf music']",1fHHq3qHU8wpRKHzhojZ4a,NaN,172066.0,False,Compelling Percussion,0.588,...,-17.278,0.0,0.0361,0.00256,0.745000,0.145,0.801,121.962,4.0,11.0
4,...Baby One More TimeBritney Spears,Britney Spears,...Baby One More Time,"['dance pop', 'pop', 'post-teen pop']",3MjUtNVVq3C8Fn0MP3zhXa,https://p.scdn.co/mp3-preview/da2134a161f1cb34...,211066.0,False,...Baby One More Time (Digital Deluxe Version),0.759,...,-5.745,0.0,0.0307,0.20200,0.000131,0.443,0.907,92.960,4.0,77.0


In [17]:
total_hot_data = pd.merge(hotstuff_data, hot100_data, how="left", on=["Performer", "Song"])
total_hot_data
total_hot_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 330470 entries, 0 to 330469
Data columns (total 30 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   url                        330470 non-null  object 
 1   WeekID                     330470 non-null  object 
 2   Week Position              330470 non-null  int64  
 3   Song                       330470 non-null  object 
 4   Performer                  330470 non-null  object 
 5   SongID_x                   330470 non-null  object 
 6   Instance                   330470 non-null  int64  
 7   Previous Week Position     298288 non-null  float64
 8   Peak Position              330470 non-null  int64  
 9   Weeks on Chart             330470 non-null  int64  
 10  SongID_y                   330235 non-null  object 
 11  spotify_genre              315727 non-null  object 
 12  spotify_track_id           287075 non-null  object 
 13  spotify_track_preview_url  16

In [18]:
total_hot_data.rename(columns = {'Performer': 'name'}, inplace = True )
total_hot_data.drop(['url', 'WeekID', 'SongID_x', 'Instance', 'Week Position', 'Previous Week Position'], axis=1, inplace=True)
total_hot_data.drop(['spotify_track_id', 'spotify_track_preview_url', 'spotify_track_duration_ms', 'spotify_track_explicit'], axis=1, inplace=True)
total_hot_data.drop(['SongID_y', 'spotify_track_album'], axis=1, inplace=True)
total_hot_data

,Song,name,Peak Position,Weeks on Chart,spotify_genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,spotify_track_popularity
0,Don't Just Stand There,Patty Duke,34,4,['deep adult standards'],0.574,0.256,7.0,-15.044,1.0,0.0298,0.6100,0.000077,0.1000,0.568,82.331,3.0,21.0
1,Don't Just Stand There,Patty Duke,22,5,['deep adult standards'],0.574,0.256,7.0,-15.044,1.0,0.0298,0.6100,0.000077,0.1000,0.568,82.331,3.0,21.0
2,Don't Just Stand There,Patty Duke,14,6,['deep adult standards'],0.574,0.256,7.0,-15.044,1.0,0.0298,0.6100,0.000077,0.1000,0.568,82.331,3.0,21.0
3,Don't Just Stand There,Patty Duke,10,7,['deep adult standards'],0.574,0.256,7.0,-15.044,1.0,0.0298,0.6100,0.000077,0.1000,0.568,82.331,3.0,21.0
4,Don't Just Stand There,Patty Duke,8,8,['deep adult standards'],0.574,0.256,7.0,-15.044,1.0,0.0298,0.6100,0.000077,0.1000,0.568,82.331,3.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330465,God Is A Woman,Ariana Grande,8,13,"['dance pop', 'pop', 'post-teen pop']",0.602,0.658,1.0,-5.934,1.0,0.0558,0.0233,0.000060,0.2370,0.268,145.031,4.0,62.0
330466,I Wanna Get Next To You,Rose Royce,10,13,"['classic soul', 'disco', 'funk', 'motown', 'p...",0.620,0.179,0.0,-19.981,1.0,0.0314,0.8350,0.030000,0.0928,0.675,130.864,4.0,51.0
330467,I Can't Stand It,Eric Clapton And His Band,10,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
330468,Here I Am Come & Take Me,Al Green,10,13,"['classic soul', 'funk', 'memphis soul', 'quie...",0.815,0.388,11.0,-12.238,0.0,0.0364,0.3180,0.000084,0.0673,0.846,94.558,4.0,52.0


In [19]:
total_hot_data.to_csv("data/hot_totals.csv")

In [20]:
top_200_na_tours_path = "data/top_200_na_tours.csv"

top_200_na_tours_data = pd.read_csv(top_200_na_tours_path)

In [21]:
top_200_na_tours_data["name"].str.strip()
top_200_na_tours_data.head(10)

,year,ranking,gross_Mil,name,avg_tick_price,avg_tick,total_tick,avg_gross,cities,shows,agency
0,2020,1,71.2,Celine Dion,174.36,12376,408407,2157894,27.0,33.0,CDA Productions
1,2020,2,58.2,Trans-Siberian Orchestra,65.76,9733,885740,640094,56.0,91.0,WME
2,2020,3,40.3,Post Malone,134.26,13042,299969,1751069,23.0,23.0,United Talent Agency
3,2020,4,33.6,Eagles,258.83,12963,129632,3355265,4.0,10.0,Creative Artists Agency
4,2020,5,26.3,Dead & Company,427.90,7692,61537,3291458,3.0,8.0,WME
5,2020,6,25.5,Andrea Bocelli,185.64,12510,137614,2322429,10.0,11.0,Klassics Music Management Limited
6,2020,7,24.3,George Strait,170.55,15822,142397,2698419,5.0,9.0,The Erv Woolsey Agency
7,2020,8,24.1,Aventura,136.72,12593,176301,1721690,6.0,14.0,Global Talent Booking
8,2020,9,22.5,Cher,124.85,11240,179838,1403270,14.0,16.0,Marshall Arts
9,2020,10,21.3,Jonas Brothers,103.75,13718,205774,1423277,14.0,15.0,United Talent Agency


In [22]:
total_tour_data = pd.merge(top_200_na_tours_data, mtv_genres_final, how="left", on=["name"])
total_tour_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1204 entries, 0 to 1203
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   year            1204 non-null   int64  
 1   ranking         1204 non-null   int64  
 2   gross_Mil       1204 non-null   float64
 3   name            1204 non-null   object 
 4   avg_tick_price  1204 non-null   float64
 5   avg_tick        1204 non-null   object 
 6   total_tick      801 non-null    object 
 7   avg_gross       1204 non-null   object 
 8   cities          801 non-null    float64
 9   shows           801 non-null    float64
 10  agency          798 non-null    object 
 11  genre           665 non-null    object 
dtypes: float64(4), int64(2), object(6)
memory usage: 122.3+ KB


In [23]:
#Identifying genres for some of the missing data
total_tour_data["genre"][0] = "Pop"
total_tour_data["genre"][14]= "Rock"
total_tour_data["genre"][20] = "Rock"
total_tour_data["genre"][21] = "World/International"
total_tour_data["genre"][2] = "Hip-Hop/Rap"
total_tour_data["genre"][4] = "Rock"
total_tour_data["genre"][23] = "Comedy"
total_tour_data["genre"][27] = "Country"
total_tour_data["genre"][17] = "Comedy"
total_tour_data["genre"][26] = "Comedy"
total_tour_data["genre"][35] = "Comedy"
total_tour_data["genre"][34] = "Comedy"
total_tour_data["genre"][29] = "World/International"
total_tour_data["genre"][31] = "Country"
total_tour_data["genre"][36] = "Comedy"
total_tour_data["genre"][1200] = "Pop"
total_tour_data["genre"][41] = "Comedy"
total_tour_data["genre"][1196] = "Pop"
total_tour_data["genre"][47] = "Metal"

total_tour_data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1204 entries, 0 to 1203
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   year            1204 non-null   int64  
 1   ranking         1204 non-null   int64  
 2   gross_Mil       1204 non-null   float64
 3   name            1204 non-null   object 
 4   avg_tick_price  1204 non-null   float64
 5   avg_tick        1204 non-null   object 
 6   total_tick      801 non-null    object 
 7   avg_gross       1204 non-null   object 
 8   cities          801 non-null    float64
 9   shows           801 non-null    float64
 10  agency          798 non-null    object 
 11  genre           683 non-null    object 
dtypes: float64(4), int64(2), object(6)
memory usage: 162.3+ KB


C:\Users\dlken\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\dlken\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\dlken\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

In [24]:
total_tour_data[total_tour_data["genre"].isna()]

,year,ranking,gross_Mil,name,avg_tick_price,avg_tick,total_tick,avg_gross,cities,shows,agency,genre
32,2020,33,6.18,Jim Gaffigan,52.22,4082,118373,213141,21.0,29.0,United Talent Agency,NaN
38,2020,39,5.36,“Hits Deep Tour” / TobyMac,36.15,6443,148187,232915,22.0,23.0,Creative Artists Agency,NaN
39,2020,40,5.24,“Drive-In Theater Tour Series”,32.38,1191,161924,38549,136.0,136.0,Awakening Events,NaN
43,2020,44,4.82,“Winter Jam” / Crowder,21.69,7165,222108,155411,31.0,31.0,Premier Productions,NaN
44,2020,45,4.64,Illenium,50.50,5746,91941,290179,14.0,16.0,United Talent Agency,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1194,2015,191,4.60,Bill Maher,66.20,1511,NaN,100000,NaN,NaN,NaN,NaN
1195,2015,192,4.60,Bill Burr,43.08,3051,NaN,131429,NaN,NaN,NaN,NaN
1197,2015,194,4.50,David Gray / Amos Lee,51.05,4198,NaN,214286,NaN,NaN,NaN,NaN
1198,2015,195,4.50,Slightly Stoopid,30.94,3030,NaN,93750,NaN,NaN,NaN,NaN


In [25]:
#Replacing remaining missing genre designations with unknown
total_tour_data['genre'] = total_tour_data['genre'].fillna("Unknown")
total_tour_data[total_tour_data["genre"].isna()]

,year,ranking,gross_Mil,name,avg_tick_price,avg_tick,total_tick,avg_gross,cities,shows,agency,genre


In [26]:
total_tour_data.to_csv("data/na_tour_final.csv")